### [안내] 
##### 본 파일은 다음과 같은 구성을 갖습니다. 
##### AI Program 동작에 있어 가장 먼저 실행되는 파일 입니다.

* 필수 라이브러리 
* 활성화 함수 및 손실함수에 따른 활성화 함수의 미분 (변수명 derv는 미분을 의미)
* sigmoid(x), sigmoid_derv(y), simoid_cross_entropy_with_logits(z, x), simoid_cross_entropy_with_logits_derv(z, x)
* relu(x), relu_derv(y)
* softmax(x), softmax_corss_entropy_with_logits(labels, logits), softmax_cross_entropy_with_logits_derv(labels, logits)

* 기타 기능
* 데이터 불러오기 load_csv(path, skip_header)
* 원-핫 벡터 처리 onehat(xs, cnt)
* 벡터 출력을 돕는 메서드 vector_to_str(x, fmt, max_cnt)
* 이미지 array 변환 메서드 load_image_pixels(imagepath, resolution, input_shape)
* 이미지 출력 메서드 draw_images_horz(xs, image_shape)
* 다중분류모델 예측여부 판단 메서드 show_select_results(est, ans, target_names, max_cnt)
* 경로 내 폴더 이름 반환 메서드 list_dir(path)

In [2]:
import numpy as np
import time
import os
import csv
import matplotlib.pyplot as plt
from PIL import Image

In [3]:

def sigmoid(x):
    return np.exp(-relu(-x)) / (1.0 + np.exp(-np.abs(x)))
        
def sigmoid_derv(y):
    return y * (1 - y)

def sigmoid_cross_entropy_with_logits(z, x):
    return relu(x) - x * z + np.log(1 + np.exp(-np.abs(x)))

def sigmoid_cross_entropy_with_logits_derv(z, x):
    return -z + sigmoid(x)

In [4]:
def relu(x):
    return np.maximum(x, 0)

def relu_derv(y):
    return np.sign(y)

In [5]:
def softmax(x):
    max_elem = np.max(x, axis=1)
    diff = (x.transpose() - max_elem).transpose()
    exp = np.exp(diff)
    sum_exp = np.sum(exp, axis=1)
    probs = (exp.transpose() / sum_exp).transpose()
    return probs

def softmax_cross_entropy_with_logits(labels, logits):
    probs = softmax(logits)
    return -np.sum(labels * np.log(probs+1.0e-10), axis=1)

def softmax_cross_entropy_with_logits_derv(labels, logits):
    return softmax(logits) - labels

In [6]:
def load_csv(path, skip_header=True):
    with open(path) as csvfile:
        csvreader = csv.reader(csvfile)
        headers = None
        if skip_header: headers = next(csvreader, None)
        rows = []
        for row in csvreader:
            rows.append(row)
            
    return rows, headers

In [7]:
def onehot(xs, cnt):
    return np.eye(cnt)[np.array(xs).astype(int)]

def vector_to_str(x, fmt='%.2f', max_cnt=0):
    if max_cnt == 0 or len(x) <= max_cnt:
        return '[' + ','.join([fmt]*len(x)) % tuple(x) + ']'
    v = x[0:max_cnt]
    return '[' + ','.join([fmt]*len(v)) % tuple(v) + ',...]'

In [8]:
def load_image_pixels(imagepath, resolution, input_shape):
    img = Image.open(imagepath)
    resized = img.resize(resolution)
    return np.array(resized).reshape(input_shape)

def draw_images_horz(xs, image_shape=None):
    show_cnt = len(xs)
    fig, axes = plt.subplots(1, show_cnt, figsize=(5,5))
    for n in range(show_cnt):
        img = xs[n]
        if image_shape:
            x3d = img.reshape(image_shape)
            img = Image.fromarray(np.uint8(x3d))
        axes[n].imshow(img)
        axes[n].axis('off')
    plt.draw()
    plt.show()

In [9]:
def show_select_results(est, ans, target_names, max_cnt=0):
    for n in range(len(est)):
        pstr = vector_to_str(100*est[n], '%2.0f', max_cnt)
        estr = target_names[np.argmax(est[n])]
        astr = target_names[np.argmax(ans[n])]
        rstr = 'O'
        if estr != astr: rstr = 'X'
        print('추정확률분포 {} => 추정 {} : 정답 {} => {}'. \
              format(pstr, estr, astr, rstr))

In [10]:
def list_dir(path):
    filenames = os.listdir(path)
    filenames.sort()
    return filenames